In [24]:
!pip install --upgrade pip
!pip install pyarrow
!pip install pandas
!pip install duckdb
!pip install hnswlib

In [25]:
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import pyarrow as pa
import duckdb
import hnswlib

In [26]:
test_table = pq.read_table('test_data_to_record.parquet')
panda_test_table = test_table.to_pandas()
# print(panda_test_table.head())

train_table = pq.read_table('train_data_to_record.parquet')
panda_train_table = train_table.to_pandas()
# print(panda_train_table.head())


In [27]:
# print(duckdb.query('''
# SELECT COUNT(*)
# FROM 'test_data_to_record.parquet'
# WHERE metadata.location ILIKE '%San Francisco%';
# ''').fetchall())
print("Record in test")
print(duckdb.query('''
SELECT COUNT(*)
FROM 'test_data_to_record.parquet'
;
''').fetchall())
print("Record in train")
print(duckdb.query('''
SELECT COUNT(*)
FROM 'train_data_to_record.parquet'
;
''').fetchall())

Record in test
[(10000,)]
Record in train
[(60000,)]


In [52]:
import hnswlib
import numpy as np

#dim = 16
# num_elements = 10000

# Generating sample data
# data = np.float32(np.random.random((num_elements, dim)))

# We split the data in two batches:
data1 = data[:num_elements // 2]
data2 = data[num_elements // 2:]

data1 = panda_test_table['embedding_data'].to_numpy().tolist()
data2 = panda_train_table['embedding_data'].to_numpy().tolist()
dim = len(data1[0])
num_elements = len(data1) + len(data2)
print("dimensionality is:", dim)
print("total number of elements is:", num_elements)
print("max elements", num_elements//2)

concatted_data = data1 + data2
print("concatted_data", len(concatted_data))

# Declaring index
p = hnswlib.Index(space='l2', dim=dim)  # possible options are l2, cosine or ip

# Initing index
# max_elements - the maximum number of elements (capacity). Will throw an exception if exceeded
# during insertion of an element.
# The capacity can be increased by saving/loading the index, see below.
#
# ef_construction - controls index search speed/build speed tradeoff
#
# M - is tightly connected with internal dimensionality of the data. Strongly affects the memory consumption (~M)
# Higher M leads to higher accuracy/run_time at fixed ef/efConstruction

p.init_index(max_elements=len(data1), ef_construction=100, M=16)

# Controlling the recall by setting ef:
# higher ef leads to better accuracy, but slower search
p.set_ef(10)

# Set number of threads used during batch search/construction
# By default using all available cores
p.set_num_threads(4)

print(data1[0])

print("Adding first batch of %d elements" % (len(data1)))
p.add_items(data1)

# Query the elements for themselves and measure recall:
labels, distances = p.knn_query(data1, k=1)
print("Recall for the first batch:", np.mean(labels.reshape(-1) == np.arange(len(data1))), "\n")

# Serializing and deleting the index:
index_path='first_half.bin'
print("Saving index to '%s'" % index_path)
p.save_index(index_path)
del p

# Reiniting, loading the index
p = hnswlib.Index(space='l2', dim=dim)  # the space can be changed - keeps the data, alters the distance function.

print("\nLoading index from 'first_half.bin'\n")

# Increase the total capacity (max_elements), so that it will handle the new data
p.load_index("first_half.bin", max_elements = num_elements)

print("Adding the second batch of %d elements" % (len(data2)))
p.add_items(data2)

# Query the elements for themselves and measure recall:
labels, distances = p.knn_query(concatted_data, k=1)
print("Recall for two batches:", np.mean(labels.reshape(-1) == np.arange(len(concatted_data))), "\n")

dimensionality is: 10
total number of elements is: 70000
max elements 35000
concatted_data 70000
[-12.8481102  -15.02010345 -10.95824623 -10.21315765 -16.63255692
 -18.05376053 -21.12276649   7.73786306 -16.47409248 -10.12139034]
Adding first batch of 10000 elements
Recall for the first batch: 0.9998 

Saving index to 'first_half.bin'

Loading index from 'first_half.bin'

Adding the second batch of 60000 elements
Recall for two batches: 0.9998428571428571 

